# Importing Libraries

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    PolynomialFeatures, 
    LabelEncoder,
    StandardScaler, 
    MinMaxScaler, 
    OneHotEncoder, 
    KBinsDiscretizer
)
from sklearn.linear_model import (
    LinearRegression, 
    LogisticRegression,
    Ridge, 
    Lasso
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    RandomForestClassifier,
)   
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error, 
    mean_squared_error, 
    r2_score, 
    confusion_matrix, 
    accuracy_score, 
    classification_report,
    recall_score,
    precision_score
)


from scripts.clean_dataset import clean_dataset

# Importing the cleaned dataset

In [2]:
df_train = pd.read_csv('data/Train_cleaned.csv')
#df_train = pd.read_csv('data/Train_cleaned_log_transformed.csv')
df_test  = pd.read_csv('data/Test.csv')
df_test  = clean_dataset(df_test)

## Categorization of total_cost (manually)

In [ ]:
bin_edges  = [0, 500, 1500, 5000, float('inf')]
bin_labels = ['Low', 'Medium', 'High', 'Very High']

# Create a new column 'cost_category' based on the binning
df_train['cost_category'] = pd.cut(
    df_train['total_cost_euro'], 
    bins=bin_edges, 
    labels=bin_labels, 
    right=False,
    include_lowest=True,
#        below=below_label  # Custom lab
)

## Categorization of total_cost (using KBins)

In [3]:
kbins = KBinsDiscretizer(
    n_bins=5,
    encode='onehot-dense', # onehot, onehoe-dense, ordinal
    strategy='quantile',   # kmeans, uniform, quantile
)
bin_data  = kbins.fit_transform(df_train[['total_cost_euro']])
bin_names = ['very low', 'low', 'medium', 'high', 'very high']
bin_edges = kbins.bin_edges_[0].round()

# Apply pd.cut with custom bin edges to start the first bin at zero
bin_edges = np.concatenate(([0], bin_edges[1:]))

df_train['total_cost_category'] = pd.cut(
    df_train['total_cost_euro'], 
    bins=bin_edges, 
    labels=bin_names, 
#    right=False,
    include_lowest=True,
)


for i in range(len(bin_names)):
    print(bin_names[i].ljust(10), f"{bin_edges[i]} - {bin_edges[i+1]}")

very low   0.0 - 218.0
low        218.0 - 828.0
medium     828.0 - 2071.0
high       2071.0 - 4766.0
very high  4766.0 - 36330.0


### Seperate numerical and categorical features

In [4]:
num_features = ['total_female', 'total_male', 'night_mainland', 'night_zanzibar', 'night_total', 'total_travelers']
#num_features = ['night_total', 'total_travelers']
#num_features = df_train.select_dtypes('number')
cat_features = list(set(df_train.columns) - set(num_features) - set(('total_cost', 'total_cost_euro')))
#cat_features = ['country', 'age_group', 'travel_with', 'purpose', 'main_activity', 'tour_arrangement', 'first_trip_tz']

### Setup Target and Features

In [5]:
#y_train = df_train['total_cost_euro']
y_train = df_train['total_cost_category']
X_train = df_train[num_features + cat_features]
X_test  = df_test#[num_features + cat_features]

# Baseline Model

Given our target variable ('total_cost') is a continuous variable, this is a regression problem. 

In [6]:
print(df_train.total_cost_euro.mode())
print(df_train.total_cost_euro.mean())

0    604.99
Name: total_cost_euro, dtype: float64
3070.1119641657333


In [7]:
df_train.total_cost_euro.describe()

count     4465.000000
mean      3070.111964
std       4522.171829
min         17.880000
25%        302.490000
50%       1337.020000
75%       3690.420000
max      36329.500000
Name: total_cost_euro, dtype: float64

In [8]:
X_train = df_train.drop(['total_cost','total_cost_euro', 'cost_category', 'total_cost_category'], axis=1)
y_train = df_train['total_cost_category']
X_test  = df_test

KeyError: "['cost_category'] not found in axis"

# Create a pipeline

### Building the pipeline

In [9]:
num_features = list(X_train.select_dtypes(include=['int64', 'float64']).columns)
cat_features = list(X_train.select_dtypes(include=['object']).columns)

In [29]:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler',  StandardScaler()),
    ('poly',    PolynomialFeatures(degree=2))
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ("num", num_transformer, num_features),
    ("cat", cat_transformer, cat_features),
])

# Create a Logistic Regression pipeline
estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),

#('classifier',   LogisticRegression(max_iter=10_000, class_weight="balanced"))
    ('classifier',   RandomForestClassifier())
])

# Fit the Logistic Regression pipeline
estimator.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('poly',
                                                                   PolynomialFeatures())]),
                                                  ['total_female', 'total_male',
                                                   'night_mainland',
                                                   'night_zanzibar',
                                                   'night_total',
                                                   'total_travelers',
                                                   'package_transport_int',
                                                   'package_sightseeing',
                                                   'package_insurance',
                                                   'first_trip_tz...
                                                   'package_accomodation',
                                                   'package_transport_tz']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['purpose', 'main_activity',
                                                   'payment_mode',
                                                   'travel_with', 'info_source',
                                                   'tour_arrangement',
                                                   'most_impressing',
                                                   'age_group', 'country'])])),
                ('classifier', RandomForestClassifier())])

In [30]:
# Predict for training and test sets
y_pred_train = estimator.predict(X_train)
y_pred_test  = estimator.predict(X_test)

train_score  = estimator.score(X_train, y_pred_train)
test_score   = estimator.score(X_test,  y_pred_test)

print("Accuracy on train set:", round(train_score,3))
print("Accuracy on test set:",  round(test_score,3))

Accuracy on train set: 1.0
Accuracy on test set: 1.0


In [31]:
y_pred_train_cv = cross_val_predict(estimator, X_train, y_train, cv=5)

In [32]:
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train), 2))
print(classification_report(y_train, y_pred_train))

#print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
train_acccuracy = estimator.score(X_train, y_pred_train)
test_acccuracy  = estimator.score(X_test, y_pred_test)
print(f"training accuracy: {round(train_acccuracy, 2)}")

print(f"test accuracy: {round(test_acccuracy, 2)}")

# Evaluate the model
# Calculating the accuracy for the LogisticRegression Classifier 
print('Cross validation scores:')
print('-------------------------')
print("Accuracy: {:.2f}".format(accuracy_score(y_train, y_pred_train)))
#print("Recall: {:.2f}".format(recall_score(y_train, y_pred_train)))
#print("Precision: {:.2f}".format(precision_score(y_train, y_pred_train)))

Accuracy on train set: 0.99
              precision    recall  f1-score   support

        high       0.99      1.00      0.99       891
         low       0.99      1.00      0.99       884
      medium       1.00      0.99      0.99       901
   very high       0.99      0.99      0.99       893
    very low       0.99      0.99      0.99       896

    accuracy                           0.99      4465
   macro avg       0.99      0.99      0.99      4465
weighted avg       0.99      0.99      0.99      4465

training accuracy: 1.0
test accuracy: 1.0
Cross validation scores:
-------------------------
Accuracy: 0.99


In [ ]:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler',  StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
]),

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
#    ('cat', cat_transformer, cat_features),
])

estimator2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier',   LogisticRegression(max_iter=10_000, class_weight="balanced"))
])

y_train_pred = cross_val_predict(estimator2, X_train, y_train, cv=5)

estimator2.fit(X_train, y_train)
y_pred_train = estimator2.predict(X_train)
y_pred_test  = estimator2.predict(X_test)

estimator2.score(X_test, y_pred_test)

In [ ]:
# Define a parameter grid for GridSearchCV
param_grid = {
    'classifier': [
        LogisticRegression(max_iter=10_000),
        Ridge(alpha=0.5),
        DecisionTreeRegressor(),
        RandomForestRegressor(),
#        XGBRegressor(),
    ],
}

# Create GridSearchCV,
grid_search = GridSearchCV(
    estimator,
    param_grid,
    cv=5,
    scoring='r2', #'neg_mean_squared_error',
    verbose=1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Make predictions\n",
y_pred_train = best_model.predict(X_train)
y_pred_test  = best_model.predict(X_test)

In [ ]:
# Evaluate the best model
print('Best Model:', best_model.named_steps['classifier'])
print('Mean Absolute Percentage Error:', round(mean_absolute_percentage_error(y_train, y_pred_train), 2))
print('Mean Absolute Error:', round(np.sqrt(mean_absolute_error(y_test, y_pred_test)), 2))
print('Mean Squared Error:', round(np.sqrt(mean_squared_error(y_test, y_pred_test)), 2))
print('R-squared:', round(r2_score(y_test, y_pred_test), 2))








def coeff_info(model):
    coeff_used = np.sum(model.coef_!=0)
    print('The model is using', coeff_used, 'out of 66 features.')
    print( "The highest coefficient has a value of:", max(model.coef_.round(3)))

    
#coeff_info(best_model)

In [ ]:
def error_analysis(y_test, y_pred_test):
    """Generated true vs. predicted values and residual scatter plot for models

    Args:
        y_test (array): true values for y_test
        y_pred_test (array): predicted values of model for y_test
    """     
    # Calculate residuals
    residuals = y_test - y_pred_test
    
    # Plot real vs. predicted values 
    fig, ax = plt.subplots(1,2, figsize=(15, 5))
    plt.subplots_adjust(right=1)
    plt.suptitle('Error Analysis')
    
    ax[0].scatter(y_pred_test, y_test, color="#FF5A36", alpha=0.7)
    ax[0].plot([-400, 350], [-400, 350], color="#193251")
    ax[0].set_title("True vs. predicted values", fontsize=16)
    ax[0].set_xlabel("predicted values")
    ax[0].set_ylabel("true values")
    ax[0].set_xlim((y_pred_test.min()-10), (y_pred_test.max()+10))
    ax[0].set_ylim((y_test.min()-40), (y_test.max()+40))
    
    ax[1].scatter(y_pred_test, residuals, color="#FF5A36", alpha=0.7)
    ax[1].plot([-400, 350], [0,0], color="#193251")
    ax[1].set_title("Residual Scatter Plot", fontsize=16)
    ax[1].set_xlabel("predicted values")
    ax[1].set_ylabel("residuals")
    ax[1].set_xlim((y_pred_test.min()-10), (y_pred_test.max()+10))
    ax[1].set_ylim((residuals.min()-10), (residuals.max()+10))
    
error_analysis(y_test, y_pred_test)

In [ ]:

# Plotting the results
plt.figure(figsize=(8, 6))

# Scatter plot of true vs. predicted values
plt.scatter(y_test, y_pred, color='blue', label='Actual vs. Predicted')

# Plot the identity line (y = x)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Identity Line')

plt.title('Regression Model: Actual vs. Predicted Values')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

# Saving the model

In [ ]:
from scripts.model_serializer import ModelSerializer

best_model_serializer = ModelSerializer('models/best_model.sav')
best_model_serializer.dump(best_model)

# Loading the model

In [ ]:
serializer = ModelSerializer('models/best_model.sav')
best_model = serializer.load()
best_model